In [1]:
import JSON

In [2]:
reaction_edges = JSON.parsefile("../kegg/2018-09-25/reaction_edges.json");

In [3]:
reactions = [i for i in keys(reaction_edges["products"])] ## could use ["substrates""] too

cpd_reactants_flat = unique(Iterators.flatten(values(reaction_edges["substrates"])))
cpd_products_flat = unique(Iterators.flatten(values(reaction_edges["products"])))
compounds = unique(Iterators.flatten([cpd_reactants_flat,cpd_products_flat]));


#### R and P: Reactant and Product matrices

In [4]:
## For R and P:
## Reactons in columns
## Compounds in rows
## Rows, Columns
## This is equivilent to R and P in segrelab's github
# R = BitArray(undef, (length(reaction_dict),length(compound_dict)))
R = zeros(Int, (length(compounds),length(reactions)))
P = zeros(Int, (length(compounds),length(reactions)));

** Below loop takes about a minute to run when using all of KEGG **

In [5]:
for (i,r) in enumerate(reactions)
    R[:,i] = [Int(cpd in reaction_edges["substrates"][r]) for cpd in compounds]
    P[:,i] = [Int(cpd in reaction_edges["products"][r]) for cpd in compounds]
end

In [6]:
R

8279×10287 Array{Int64,2}:
 1  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  1  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  1  1
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  1  0  1  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  1     0  0  0  1  0  0  0  0  0  0  1  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  

In [7]:
## Check to make sure R being populated correctly
# length(collect(Iterators.flatten(values(reaction_edges["substrates"]))))==length(findall(R))


In [8]:
## Check to make sure P being populated correctly
# length(collect(Iterators.flatten(values(reaction_edges["products"]))))==length(findall(P))


#### x: seed compounds vector

In [9]:
# seed_compounds = JSON.parsefile("../seeds.json");
# x = [Int(i in seed_compounds["Enceladus_20-SAFR-032"]) for i in compounds];

# ds80_seeds = ["C00011",
# "C20298",
# "C14819",
# "C00087",
# "C00237",
# "C00058",
# "C00033",
# "C00031",
# "C00095",
# "C00124",
# "C00159",
# "C00243",
# "C00208",
# "C00282",
# "C00007",
# "C00001"]

ds80_seeds = ["C00031","C00001"]

x = [Int(i in ds80_seeds) for i in compounds];


#### t: target compounds vector

In [10]:
target_compounds = JSON.parsefile("../targets/Freilich09.json");
target_compounds_org = collect(intersect(keys(target_compounds),compounds));
t = [Int(i in target_compounds_org) for i in compounds];

## Make the network expansion loop!!!

In [44]:
function netexp(R::Array{Int,2},P::Array{Int,2},x::Array{Int,1})
    
    # initialize variables:
    # find the total number of metabolites in the seed set
    k = sum(x);
    # initialize previous iteration count of metabolites in the network
    k0 = 0;
    # iteration 1 consistes of the seed set
    X = []
    push!(X,x)  ## Each row is 1 generation
    # initialize reaction accumulation matrix
    Y = [];
    
    # transpose R
    RT = transpose(R)
    PT = transpose(P)
    
    b = [sum(RT[i,:]) for i in 1:size(RT)[1]]
    bp = [sum(PT[i,:]) for i in 1:size(PT)[1]]

    # while the metabolite set has not converged
    while k > k0 
        
        # update previous number of metabolites
        k0 = k;
        
        # RT*x ==> represnts the number of present metabolites in the 
        # network within each reaction; if this isequal to the total 
        # number of metabolites in each reaction, then the reaction 
        # is added to the network
        y = Array{Int}(RT * x .== b)  ## Forward reactions
        yp = Array{Int}(PT * x .== bp) ## Backward reactions

        # P*y > 0 ==> represents the vector of reactions that produce 
        # metabolite i. (i in 1:m).  If this is >0, 
        # then that metabolite is producable 
        xnew = Array{Int}(P * y .> 0)   ## Forward reactions
        xnewp = Array{Int}(R * yp .> 0) ## Backward reactions
        
        #add to previous set of metabolites (only needed to retain seed set)
        x = x .| xnew .| xnewp ## Add forward and backward reactions
        
        # find new total number of metabolites in network
        k = sum(x);

        # append accumulation matricies
        push!(X,x)
        push!(Y,y .| yp)
    
    end
    (X,Y)
end
    

netexp (generic function with 1 method)

In [45]:
(X,Y) = netexp(R,P,x)

(Any[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  …  [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0

In [46]:
size(X)

(26,)

In [54]:
size(Y)

(25,)

In [55]:
push!(Y,Y[end])

26-element Array{Any,1}:
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [56]:
X

26-element Array{Any,1}:
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [57]:
for i in 1:length(X)
    println(sum(X[i]))
end

2
6
19
30
36
41
48
66
86
118
150
176
203
219
230
240
246
255
264
271
275
281
285
286
288
288


In [64]:
length(X)

26

In [63]:
X[1:end-1]

25-element Array{Any,1}:
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 1, 0, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [59]:
pop!(X,X[end])

MethodError: MethodError: no method matching pop!(::Array{Any,1}, ::Array{Int64,1})
Closest candidates are:
  pop!(::Array{T,1} where T) at array.jl:1064
  pop!(!Matched::IdDict{K,V}, ::Any) where {K, V} at abstractdict.jl:619
  pop!(!Matched::IdDict{K,V}, ::Any, !Matched::Any) where {K, V} at abstractdict.jl:607
  ...

In [58]:
for i in 1:length(Y)
    println(sum(Y[i]))
end

4
18
39
46
54
60
78
100
138
183
213
247
266
278
289
297
308
317
326
330
337
342
343
345
345
345


In [53]:
sum(Y[end])

345

In [29]:
# unique([Y[1][i]==1 ? reactions[i] : "SKIP" for (i,r) in enumerate(reactions)])


In [30]:
length(reactions)

10287

In [31]:
length(compounds)

8279

In [32]:
reactions_by_generation = Dict()
for j in 1:size(Y)[1]
    reactions_by_generation[j] = []
    for (i,r) in enumerate(reactions)
        if Y[j][i]==1
            push!(reactions_by_generation[j],reactions[i])
        end
    end
end

In [33]:
compounds_by_generation = Dict()
for j in 1:size(X)[1]
    compounds_by_generation[j] = []
    for (i,c) in enumerate(compounds)
        if X[j][i]==1
            push!(compounds_by_generation[j],compounds[i])
        end
    end
end

In [34]:
reactions_by_generation

Dict{Any,Any} with 18 entries:
  18 => Any["R03857", "R02250", "R08640", "R07506", "R10331", "R00340", "R02433…
  2  => Any["R00340", "R09186", "R06699", "R09311", "R02782", "R00363", "R05771…
  16 => Any["R03857", "R02250", "R08640", "R07506", "R10331", "R00340", "R02433…
  11 => Any["R03857", "R02250", "R08640", "R07506", "R10331", "R00340", "R02433…
  7  => Any["R03857", "R02250", "R08640", "R10331", "R00340", "R02433", "R02769…
  9  => Any["R03857", "R02250", "R08640", "R07506", "R10331", "R00340", "R02433…
  10 => Any["R03857", "R02250", "R08640", "R07506", "R10331", "R00340", "R02433…
  17 => Any["R03857", "R02250", "R08640", "R07506", "R10331", "R00340", "R02433…
  8  => Any["R03857", "R02250", "R08640", "R07506", "R10331", "R00340", "R02433…
  6  => Any["R03857", "R02250", "R08640", "R10331", "R00340", "R02433", "R02769…
  4  => Any["R03857", "R00340", "R02433", "R02769", "R09186", "R00743", "R10908…
  3  => Any["R00340", "R02769", "R09186", "R00743", "R06699", "R07650", "R0962

In [35]:
compounds_by_generation

Dict{Any,Any} with 19 entries:
  18 => Any["C01832", "C00016", "C00422", "C00001", "C17224", "C00024", "C15778…
  2  => Any["C00001", "C02107", "C03826", "C12366", "C00029", "C03752", "C18357…
  16 => Any["C01832", "C00016", "C00422", "C00001", "C17224", "C00024", "C15778…
  11 => Any["C01832", "C00016", "C00422", "C00001", "C17224", "C00024", "C15778…
  7  => Any["C01832", "C00016", "C00422", "C00001", "C17224", "C00024", "C00005…
  9  => Any["C01832", "C00016", "C00422", "C00001", "C17224", "C00024", "C15778…
  10 => Any["C01832", "C00016", "C00422", "C00001", "C17224", "C00024", "C15778…
  19 => Any["C01832", "C00016", "C00422", "C00001", "C17224", "C00024", "C15778…
  17 => Any["C01832", "C00016", "C00422", "C00001", "C17224", "C00024", "C15778…
  8  => Any["C01832", "C00016", "C00422", "C00001", "C17224", "C00024", "C15778…
  6  => Any["C01832", "C00016", "C00422", "C00001", "C17224", "C00024", "C00005…
  4  => Any["C01832", "C00016", "C00001", "C00024", "C00005", "C00080", "C0000